In [4]:
using JuMP, JuMPeR, Gurobi, LinearAlgebra

In [5]:
function solve_stupid_model()

    # Get problem data.
    num_partitions = 2
    jobsStart = [50,233,233]
    jobsEnd = [150,350,350]
    num_jobs = 3
    num_workers = 2
    num_intervals = 2
    cost_backup = 10
    M = 10000

    # Create model.
    m = RobustModel(solver=GurobiSolver(OutputFlag=0))

    ### Static

    # Epigraph variable.
    @variable(m, r)

    # Assignment variables for workers to jobs.
    @variable(m, x[1:num_workers, 1:num_jobs], Bin)

    # Backup agent variables.
    @variable(m, z[1:num_jobs], Bin)

    ## Adjustable

    # Workers to jobs.
    @variable(m, x_u[1:num_partitions, 1:num_workers, 2:num_intervals, 1:num_jobs], Bin)

    # Backup agents.
    @variable(m, z_u[1:num_partitions, 2:num_intervals, 1:num_jobs], Bin)

    # Variables to capture clashes.
    @variable(m, y_u[1:num_partitions, 1:num_jobs, 1:num_jobs], Bin)
 
    ## Uncertainty

    # One uncertain vector for each partition.
    @uncertain(m, u1[1:num_jobs])
    @constraint(m, norm(u1,1) <= 200)
    @constraint(m, norm(u1,Inf) <= 100)
    #apply_constraints!(m, u1, base)
    #apply_constraints!(m, u1, partitions[1])
    @constraint(m, 2*u1[1] - u1[2] <=50)
    
    @uncertain(m, u2[1:num_jobs])
    @constraint(m, norm(u2,1) <= 200)
    @constraint(m, norm(u2,Inf) <= 100)
    #apply_constraints!(m, u2, base)
    #apply_constraints!(m, u2, partitions[2])
    @constraint(m, -2*u1[1] + u1[2] <=-50)


    # Constraint on the tasks not assigned during interval t.
    initial_jobs = [1,2,3]
    @constraint(m, z[initial_jobs] .== 0)
    for i in 1:num_workers

        # Jobs in interval 1 are static.
        @constraint(m, x[i, initial_jobs] .== 0)

        # Remaining jobs are adjustable.
        for t in 2:num_intervals, p in 1:num_partitions
            interval_jobs = [1]
            #@constraint(m, x_u[p, i, t, :][interval_jobs] .== 0)
            #@constraint(m, z_u[p, t, :][interval_jobs] .== 0)
        end
    end

    # Objective function.
    @objective(m, Min, r)
    for p in 1:num_partitions
        @constraint(m,  r >= cost_backup * sum(z_u[p, 2, 2] + z_u[p, 2, 3]+z_u[p, 2, 1]))
    end

    # Convering constraints for first stage.
    for j in [1]
        #@constraint(m, sum(x[i, j] for i in 1:num_workers) + z[j] == 1)
    end

    # Covering constraints for adjustable stages.
    for t in 2:num_intervals, p in 1:num_partitions
        for j in [1,2,3]
            @constraint(m, sum(x_u[p, i, t, j] for i in 1:num_workers) + z_u[p, t, j] == 1)
        end
    end

    # Constraint chaining.
    for j in 1:num_jobs, k in 1:num_jobs
        if j != k
            j_end = jobsEnd[j]
            k_start = jobsStart[k]
            @constraint(m, - M * y_u[1, j, k] <= k_start + u1[k] - (j_end + u1[j]))
            # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
        end
    end
    
    # Constraint chaining.
    for j in 1:num_jobs, k in 1:num_jobs
        if j != k
            j_end = jobsEnd[j]
            k_start = jobsStart[k]
            @constraint(m, - M * y_u[2, j, k] <= k_start + u2[k] - (j_end + u2[j]))
            # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
        end
    end

    # No clash for initial jobs with any other jobs.
    for i in 1:num_workers
        for j in 1:num_jobs, k in (j + 1):num_jobs, p in 1:num_partitions

            # Clashes of initial jobs with other initial jobs.
            @constraint(m, x[i, j] + x[i, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))

            # Clashes of initial jobs with adjustable jobs.
            for t in 2:num_intervals
                @constraint(m, x[i, j] + x_u[p, i, t, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    # No clash for adjustable jobs.
    for i in 1:num_workers
        for t in 2:num_intervals, t_bar in t:num_intervals, p in 1:num_partitions
            for j in 1:num_jobs, k in (j + 1):num_jobs
                @constraint(m, x_u[p, i, t, j] + x_u[p, i, t_bar, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    solve(m)
    
    return m
end

solve_stupid_model (generic function with 1 method)

In [6]:
solved = solve_stupid_model()
println(getobjectivevalue(solved))

Academic license - for non-commercial use only
10.0
